<a href="https://colab.research.google.com/github/omkarade/A-to-Z-Approach-on-Time-Series-And-Forecasting./blob/main/hyperparameter_tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c store-sales-time-series-forecasting
! unzip store-sales-time-series-forecasting.zip

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
import pandas as pd
import numpy as np 
from sklearn.metrics import mean_squared_error
import pylab 
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score 
import pylab 
from math import sqrt
from joblib import Parallel, delayed
import joblib
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tran=pd.read_csv('train.csv')
stores=pd.read_csv('stores.csv')
transactions=pd.read_csv('transactions.csv')
oil=pd.read_csv('oil.csv')
holidays_events=pd.read_csv('holidays_events.csv')
data1 = tran.merge(stores, how = 'left', on = 'store_nbr' )
data1['date'] = pd.to_datetime(data1['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
oil['date'] = pd.to_datetime(oil['date'])
data2 = data1.merge(holidays_events, how = 'left', on = 'date' )
data2 = data2.merge(oil, how ='left', on ='date')
data2["type_y"] = data2["type_y"].fillna("Not Holiday")
data2["locale"] = data2["locale"].fillna("Not Holiday")
data2["locale_name"] = data2["locale_name"].fillna("Ecuador")
data2["description"] = data2["description"].fillna("NA")
data2["transferred"] = data2["transferred"].fillna(False)
data2["dcoilwtico"] = data2["dcoilwtico"].fillna(np.mean(data2["dcoilwtico"]))
data2["description"] = data2["description"].fillna("NA")
data2["month"] = data2['date'].map(lambda x: x.month)

In [ ]:
data2=data2.drop(columns="dcoilwtico")
data2=data2.drop(columns="id")
data2=data2.drop(columns="transferred")

In [ ]:
train=data2[:28000]
trainn=train[:22000]
testn=train[22000:]

In [ ]:
def clen_to_pred(data2,fulldata,model="hyperparameter_tuning"):
#training RandomForestRegressor Model
  if model=="hyperparameter_tuning":
    descp=pd.concat([data2,fulldata], ignore_index=True, axis=0)
    descp=descp.drop(columns="date")
    catage=descp[["family","city","state","type_x","type_y","locale","locale_name","description"]]
    #Converting Categorical Features Into One Hot Encoding  
    descr=pd.get_dummies(catage)
    descr["store_nbr"]=descp["store_nbr"]
    descr["onpromotion"]=descp["onpromotion"]
    descr["cluster"]=descp["cluster"]

    #Adding lag Features
    train=descr[:22000]
    train["sales"]=fulldata["sales"]
    train['sales_lag_15'] = train['sales'].shift(2)
    train['sales_lag_30'] = train['sales'].shift(3)
    train['sales_lag_60'] = train['sales'].shift(7)
    train['sales_lag_140'] = train['sales'].shift(15)
    train['sales_lag_15'] = train['sales_lag_15'].fillna(0)
    train['sales_lag_30'] = train['sales_lag_30'].fillna(0)
    train['sales_lag_60'] = train['sales_lag_60'].fillna(0)
    train['sales_lag_140'] = train['sales_lag_140'].fillna(0)
    train['Expo_moving_sale_2']=train['sales'].ewm(com=0.2).mean()
    train['Expo_moving_sale_4']=train['sales'].ewm(com=0.4).mean()
    train['Expo_moving_sale_6']=train['sales'].ewm(com=0.6).mean()
    train['Expo_moving_sale_8']=train['sales'].ewm(com=0.8).mean()
    
    #train=train.drop(columns="sales")
    y_main=fulldata.sales
    train=train.drop(columns="sales")
    train=train.fillna(0)
    train1=train[:22000]
  return train1,y_main

In [ ]:
train=data2[:28000]
trainn=train[:22000]
testn=train[22000:]
df_featl,df_targetl=clen_to_pred(testn,trainn)

In [ ]:
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split(df_featl, np.ravel(df_targetl),test_size = 0.30, random_state = 101)


**hyperparameter_tuning :-svm**

In [ ]:
from sklearn.svm import SVR
import numpy as np
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split

n_samples, n_features = 10, 5
np.random.seed(0)
y = np.random.randn(n_samples)
X = np.random.randn(n_samples, n_features)
parameters = {'kernel': ('rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7],'epsilon':[0.1,0.5]}
svr = svm.SVR()
clf = GridSearchCV(svr, parameters)
clf.fit(X_train,y_train)
clf.best_params_


{'C': 10, 'epsilon': 0.5, 'gamma': 1e-07, 'kernel': 'rbf'}

**hyperparameter_tuning :- DecisionTreeRegressor**

In [ ]:
from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
for max_d in range(1,15):
  model = DecisionTreeRegressor(max_depth=max_d, random_state=42)
  model.fit(X_train, y_train)
  print('The Training Accuracy for max_depth {} is:'.format(max_d), model.score(X_train, y_train))
  print('')


The Training Accuracy for max_depth 1 is: 0.6340521095132865

The Training Accuracy for max_depth 2 is: 0.878538025026294

The Training Accuracy for max_depth 3 is: 0.9577874304555287

The Training Accuracy for max_depth 4 is: 0.9791129899294101

The Training Accuracy for max_depth 5 is: 0.9900729418500652

The Training Accuracy for max_depth 6 is: 0.9958118296899233

The Training Accuracy for max_depth 7 is: 0.9980045516596512

The Training Accuracy for max_depth 8 is: 0.9990677491779041

The Training Accuracy for max_depth 9 is: 0.9994683547905004

The Training Accuracy for max_depth 10 is: 0.9997197405380636

The Training Accuracy for max_depth 11 is: 0.9998536251208784

The Training Accuracy for max_depth 12 is: 0.9999204492504445

The Training Accuracy for max_depth 13 is: 0.9999558545865729

The Training Accuracy for max_depth 14 is: 0.9999774278102842



**hyperparameter_tuning :- RandomForestRegressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFReg = RandomForestRegressor(n_estimators = 500, random_state = 1, n_jobs = -1) 

param_grid = { 
    'max_features' : ["sqrt", "log2"],
    'min_samples_split' : np.linspace(0.1, 1.0, 10),
     'max_depth' : [x for x in range(1,8)]

}

from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=RFReg, param_grid=param_grid, cv= 10, n_jobs = -1)
CV_rfc.fit(X_train, y_train)


GridSearchCV(cv=10,
             estimator=RandomForestRegressor(n_estimators=500, n_jobs=-1,
                                             random_state=1),
             n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])})

In [ ]:
CV_rfc.best_params_


{'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 0.1}